# Mediator with events

> Combining the Observer and Mediator patterns

We briefly discussed **events** in the *Chain of Responsibility* lesson; we defined an event as as a list of functions what we can call. Events will be discussed further in the upcoming *Observer* lesson, but we will use the same implementation we used previously to show a variation of the Mediator design pattern.

In [1]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

In this scenario we will model a soccer match. The `Game` will be our **mediator**; it will generate events that subsequently players, coaches and viewers can subscribe to and get information about something happening in the game. We will have a single `events` object that everyone can subscribe to, and we'll define a method to fire the events.

In [2]:
class Game:
    def __init__(self):
        self.events = Event()

    def fire(self, args):
        self.events(args)

We will now generate a class for informing the subscribers that a goal has been scored:

In [3]:
class GoalScoredInfo:
    def __init__(self, who_scored, goals_scored):
        self.goals_scored = goals_scored
        self.who_scored = who_scored

And we will now define the `Player` class. A `Player` is able to score goals. We will use our `Game` mediator for informing the subscribers about goal scoring.

In [4]:
class Player:
    def __init__(self, name, game):
        self.name = name
        self.game = game
        self.goals_scored = 0

    def score(self):
        self.goals_scored += 1
        args = GoalScoredInfo(self.name, self.goals_scored)
        self.game.fire(args) # we trigger a goal event

Let's now define another subscriber, `Coach`. A `Coach` subscribes to the game events and listens for goals; they will celebrate the first and second goals of a player but will ignore the player from the 3rd goal onwards.

In [5]:
class Coach:
    def __init__(self, game):
        game.events.append(self.celebrate_goal) # subscribing to the events

    def celebrate_goal(self, args):
        if isinstance(args, GoalScoredInfo) and args.goals_scored < 3:
            print(f'Coach says: well done, {args.who_scored}!')

Here's what's happening:

1. Remember that `game.events` is a list of callable functions. When we initialize `Coach`, we append the `celebrate_goal` method to that list (we call this "subscribing to the events").
2. When a `Player` scores a goal, they create a `GoalScoredInfo` instance that is sent as an argument to our events list; the `game.fire` method will trigger an event: it will loop through our list of callable functions and call each one with whatever arguments we pass, which is `GoalScoredInfo` in this case.
3. As the list of callable functions iterates, it eventually calls `celebrate_goal` and passes `GoalScoredInfo` as an argument.

Let's see our code in action:

In [6]:
game = Game()
player = Player('Sam', game)
coach = Coach(game)

player.score()  # Coach says: well done, Sam!
player.score()  # Coach says: well done, Sam!
player.score()  # ignored by coach

Coach says: well done, Sam!
Coach says: well done, Sam!
